<a href="https://colab.research.google.com/github/mjiii25/posco-academy/blob/main/Big-Data/Assignment-2/vitalfew-DT-RF-GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [2]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

### **데이터 구성하기**

In [3]:
df_raw = pd.read_excel('/content/mydrive/MyDrive/1. 실습데이터 설명(Data Dictionary).xlsx',
                       sheet_name = 'SCALE불량')

print(df_raw.shape)
df_raw.head()

(720, 21)


,PLATE_NO,ROLLING_DATE,SCALE,SPEC,STEEL_KIND,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,...,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,PB562774,2008-08-01:00:00:15,양품,AB/EH32-TM,T1,32.25,3707.0,15109.0,14180.0,1호기,...,1144.0,116.0,1133.0,59.0,282.0,1133.0,934.0,적용,8.0,2조
1,PB562775,2008-08-01:00:00:16,양품,AB/EH32-TM,T1,32.25,3707.0,15109.0,14180.0,1호기,...,1144.0,122.0,1135.0,53.0,283.0,1135.0,937.0,적용,8.0,2조
2,PB562776,2008-08-01:00:00:59,양품,NV-E36-TM,T8,33.27,3619.0,19181.0,18130.0,2호기,...,1129.0,116.0,1121.0,55.0,282.0,1121.0,889.0,적용,8.0,3조
3,PB562777,2008-08-01:00:01:24,양품,NV-E36-TM,T8,33.27,3619.0,19181.0,18130.0,2호기,...,1152.0,125.0,1127.0,68.0,316.0,1127.0,885.0,적용,8.0,3조
4,PB562778,2008-08-01:00:01:44,양품,BV-EH36-TM,T8,38.33,3098.0,13334.0,12430.0,3호기,...,1140.0,134.0,1128.0,48.0,314.0,1128.0,873.0,적용,8.0,1조


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLATE_NO           720 non-null    object 
 1   ROLLING_DATE       720 non-null    object 
 2   SCALE              720 non-null    object 
 3   SPEC               720 non-null    object 
 4   STEEL_KIND         720 non-null    object 
 5   PT_THK             720 non-null    float64
 6   PT_WDTH            720 non-null    float64
 7   PT_LTH             720 non-null    float64
 8   PT_WGT             720 non-null    float64
 9   FUR_NO             720 non-null    object 
 10  FUR_NO_ROW         720 non-null    float64
 11  FUR_HZ_TEMP        720 non-null    float64
 12  FUR_HZ_TIME        720 non-null    float64
 13  FUR_SZ_TEMP        720 non-null    float64
 14  FUR_SZ_TIME        720 non-null    float64
 15  FUR_TIME           720 non-null    float64
 16  FUR_EXTEMP         720 non

**범주형 변수의 범주 확인하기**

In [5]:
df_raw['SPEC'].value_counts()

JS-SM490YB    84
LR-A          54
NV-A          48
PILAC-BT33    40
BV-EH36-TM    35
              ..
KR-B           1
BV-A           1
CCS-B          1
KS-HSB600      1
BV-AH36-TM     1
Name: SPEC, Length: 66, dtype: int64

In [6]:
df_raw['STEEL_KIND'].value_counts()

C0    503
T8     95
T5     43
T7     35
T1     18
T0     16
C3      7
T3      2
C1      1
Name: STEEL_KIND, dtype: int64

In [7]:
df_raw['FUR_NO'].value_counts()

3호기    243
1호기    240
2호기    237
Name: FUR_NO, dtype: int64

In [8]:
df_raw['FUR_NO_ROW'].value_counts()

1.0    362
2.0    358
Name: FUR_NO_ROW, dtype: int64

In [9]:
df_raw['FUR_NO_ROW'] = df_raw['FUR_NO_ROW'].astype('object')

In [10]:
df_raw['HSB'].value_counts()

적용     687
미적용     33
Name: HSB, dtype: int64

In [11]:
df_raw['WORK_GR'].value_counts()

4조    194
1조    189
3조    172
2조    165
Name: WORK_GR, dtype: int64

**결측값 확인하기**

In [13]:
df_raw.isnull().sum()

PLATE_NO             0
ROLLING_DATE         0
SCALE                0
SPEC                 0
STEEL_KIND           0
PT_THK               0
PT_WDTH              0
PT_LTH               0
PT_WGT               0
FUR_NO               0
FUR_NO_ROW           0
FUR_HZ_TEMP          0
FUR_HZ_TIME          0
FUR_SZ_TEMP          0
FUR_SZ_TIME          0
FUR_TIME             0
FUR_EXTEMP           0
ROLLING_TEMP_T5      0
HSB                  0
ROLLING_DESCALING    0
WORK_GR              0
dtype: int64

**목표변수와 설명변수 분리하기**

In [14]:
df_raw_x = df_raw.drop('SCALE', axis = 1, inplace = False)
df_raw_y = df_raw['SCALE']

In [15]:
df_raw_y.value_counts()

양품    489
불량    231
Name: SCALE, dtype: int64

**불필요한 설명변수 제외하기 & 더미변수 생성하기**

In [16]:
df_raw_x.columns

Index(['PLATE_NO', 'ROLLING_DATE', 'SPEC', 'STEEL_KIND', 'PT_THK', 'PT_WDTH',
       'PT_LTH', 'PT_WGT', 'FUR_NO', 'FUR_NO_ROW', 'FUR_HZ_TEMP',
       'FUR_HZ_TIME', 'FUR_SZ_TEMP', 'FUR_SZ_TIME', 'FUR_TIME', 'FUR_EXTEMP',
       'ROLLING_TEMP_T5', 'HSB', 'ROLLING_DESCALING', 'WORK_GR'],
      dtype='object')

In [17]:
df_raw_x = df_raw_x.drop(['PLATE_NO', 'ROLLING_DATE', 'SPEC'], axis = 1, inplace = False)

In [18]:
df_raw_x_dummy = pd.get_dummies(df_raw_x)

print(df_raw_x_dummy.shape)
df_raw_x_dummy.head()

(720, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1.0,FUR_NO_ROW_2.0,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
0,32.25,3707.0,15109.0,14180.0,1144.0,116.0,1133.0,59.0,282.0,1133.0,...,0,0,1,0,0,1,0,1,0,0
1,32.25,3707.0,15109.0,14180.0,1144.0,122.0,1135.0,53.0,283.0,1135.0,...,0,0,0,1,0,1,0,1,0,0
2,33.27,3619.0,19181.0,18130.0,1129.0,116.0,1121.0,55.0,282.0,1121.0,...,1,0,1,0,0,1,0,0,1,0
3,33.27,3619.0,19181.0,18130.0,1152.0,125.0,1127.0,68.0,316.0,1127.0,...,1,0,0,1,0,1,0,0,1,0
4,38.33,3098.0,13334.0,12430.0,1140.0,134.0,1128.0,48.0,314.0,1128.0,...,0,1,1,0,0,1,1,0,0,0


In [19]:
df_raw_x_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PT_THK             720 non-null    float64
 1   PT_WDTH            720 non-null    float64
 2   PT_LTH             720 non-null    float64
 3   PT_WGT             720 non-null    float64
 4   FUR_HZ_TEMP        720 non-null    float64
 5   FUR_HZ_TIME        720 non-null    float64
 6   FUR_SZ_TEMP        720 non-null    float64
 7   FUR_SZ_TIME        720 non-null    float64
 8   FUR_TIME           720 non-null    float64
 9   FUR_EXTEMP         720 non-null    float64
 10  ROLLING_TEMP_T5    720 non-null    float64
 11  ROLLING_DESCALING  720 non-null    float64
 12  STEEL_KIND_C0      720 non-null    uint8  
 13  STEEL_KIND_C1      720 non-null    uint8  
 14  STEEL_KIND_C3      720 non-null    uint8  
 15  STEEL_KIND_T0      720 non-null    uint8  
 16  STEEL_KIND_T1      720 non

In [20]:
df_raw_x_dummy.describe().round(3)

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1.0,FUR_NO_ROW_2.0,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
count,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,...,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000
mean,29.268,2807.128,34323.269,41481.778,1155.339,88.528,1149.667,78.803,300.564,1149.667,...,0.329,0.338,0.503,0.497,0.046,0.954,0.262,0.229,0.239,0.269
std,19.153,517.840,13828.435,24751.322,20.866,37.661,17.572,36.421,45.390,17.572,...,0.470,0.473,0.500,0.500,0.209,0.209,0.440,0.421,0.427,0.444
min,12.030,1800.000,7893.000,7820.000,1103.000,43.000,1113.000,35.000,221.000,1113.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,16.090,2466.000,22663.000,17757.500,1139.000,65.000,1132.000,57.750,257.000,1132.000,...,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
50%,20.130,2725.000,37380.000,36120.000,1158.000,75.000,1155.000,67.500,304.000,1155.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
75%,40.000,3125.000,43735.000,61120.000,1171.000,101.000,1164.000,86.000,342.000,1164.000,...,1.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,0.000,1.000
max,100.340,4575.000,54917.000,115080.000,1206.000,312.000,1185.000,294.000,398.000,1185.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [21]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x_dummy,
                                                                df_raw_y,
                                                                test_size = 0.3)

print('Shape of df_train_x : {}'.format(df_train_x.shape))
print('Shape of df_test_x : {}'.format(df_test_x.shape))
print('Shape of df_train_y : {}'.format(df_train_y.shape))
print('Shape of df_test_y : {}'.format(df_test_y.shape))

Shape of df_train_x : (504, 32)
Shape of df_test_x : (216, 32)
Shape of df_train_y : (504,)
Shape of df_test_y : (216,)


In [22]:
print(df_train_x.shape)
df_train_x.head()

(504, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1.0,FUR_NO_ROW_2.0,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
712,16.60,3528.0,32323.0,14860.0,1156.0,65.0,1160.0,72.0,244.0,1160.0,...,0,1,0,1,0,1,1,0,0,0
79,75.71,2505.0,9249.0,13770.0,1142.0,82.0,1131.0,69.0,314.0,1131.0,...,0,0,0,1,1,0,0,0,0,1
367,15.38,3274.0,36025.0,14240.0,1196.0,60.0,1172.0,60.0,252.0,1172.0,...,1,0,0,1,0,1,0,1,0,0
285,25.18,2881.0,40423.0,46040.0,1144.0,63.0,1159.0,58.0,265.0,1159.0,...,0,1,0,1,0,1,0,0,1,0
595,14.58,2615.0,54862.0,32840.0,1168.0,58.0,1158.0,58.0,342.0,1158.0,...,1,0,1,0,0,1,0,0,0,1


In [23]:
print(df_train_y.shape)
df_train_y.head()

(504,)


712    양품
79     불량
367    양품
285    양품
595    양품
Name: SCALE, dtype: object

In [24]:
print(df_test_x.shape)
df_test_x.head()

(216, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1.0,FUR_NO_ROW_2.0,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
476,70.10,2150.0,15340.0,18150.0,1108.0,138.0,1117.0,78.0,362.0,1117.0,...,0,1,1,0,0,1,0,0,1,0
200,30.13,2288.0,30730.0,33260.0,1154.0,72.0,1155.0,85.0,276.0,1155.0,...,1,0,1,0,0,1,0,0,1,0
589,16.09,2655.0,52841.0,35440.0,1169.0,77.0,1159.0,58.0,342.0,1159.0,...,1,0,1,0,0,1,0,0,1,0
596,13.06,2750.0,54906.0,61920.0,1167.0,66.0,1158.0,84.0,342.0,1158.0,...,1,0,0,1,0,1,0,1,0,0
602,18.15,2281.0,40770.0,53000.0,1163.0,73.0,1155.0,86.0,342.0,1155.0,...,1,0,0,1,0,1,0,1,0,0


In [25]:
print(df_test_y.shape)
df_test_y.head()

(216,)


476    양품
200    불량
589    양품
596    양품
602    양품
Name: SCALE, dtype: object

### **변수 중요도 확인하기**

#### **의사결정나무(Decision Tree)**

In [26]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(df_train_x, df_train_y)

print('Accuracy on train set : {:.3f}'.format(dt_classifier.score(df_train_x, df_train_y)))
print('Accuracy on test set : {:.3f}'.format(dt_classifier.score(df_test_x, df_test_y)))

Accuracy on train set : 1.000
Accuracy on test set : 0.940


In [27]:
dt_importance = pd.DataFrame()
dt_importance['Feature'] = df_train_x.columns
dt_importance['Importance'] = dt_classifier.feature_importances_

dt_importance.sort_values('Importance', ascending = False, inplace = True)
dt_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.524
27,HSB_적용,0.186
9,FUR_EXTEMP,0.151
11,ROLLING_DESCALING,0.071
7,FUR_SZ_TIME,0.014


**<모델 성능>**
- 기본 의사결정나무는 train과 test 데이터에 대해 각각 100%, 99.5%의 성능을 보여주고 있다. 
- Train 데이터에 있어서 과대적합인 것으로 확인된다.


**<변수 중요도>**
- 의사결정나무 모델을 통해 변수 중요도를 추출해보았다. 
- ROLLING_TEMP_T5가 0.592, HSB_미적용이 0.151의 변수 중요도를 보여주고 있다.

#### **랜덤 포레스트 (Random Forest)**

In [28]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(rf_classifier.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(rf_classifier.score(df_test_x, df_test_y)))

Model Score on train set : 1.000
Model Score on test set : 0.963


In [29]:
rf_importance = pd.DataFrame()
rf_importance['Feature'] = df_train_x.columns
rf_importance['Importance'] = rf_classifier.feature_importances_

rf_importance.sort_values('Importance', ascending = False, inplace = True)
rf_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.294
9,FUR_EXTEMP,0.092
6,FUR_SZ_TEMP,0.078
26,HSB_미적용,0.066
27,HSB_적용,0.061


**<모델 성능>**
- 기본 랜덤포레스트 예측 모델은 train과 test 데이터에 대해 각각 100%, 96.8%의 설명력을 갖는다.
- 의사결정나무에 비해서는 과대적합의 경향성이 적어보이지만, 여전히 모델의 일반화 가능성은 높아 보이지 않는다.

**<변수 중요도>**
- ROLLING_TEMP_T5가 0.305, FUR_SZ_TEMP가 0.081의 변수 중요도를 가진다.

#### **그래디언트 부스팅 (Gradient Boosting)**

In [30]:
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(gb_classifier.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(gb_classifier.score(df_test_x, df_test_y)))

Model Score on train set : 1.000
Model Score on test set : 0.986


In [31]:
gb_importance = pd.DataFrame()
gb_importance['Feature'] = df_train_x.columns
gb_importance['Importance'] = gb_classifier.feature_importances_

gb_importance.sort_values('Importance', ascending = False, inplace = True)
gb_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.533
26,HSB_미적용,0.129
9,FUR_EXTEMP,0.103
11,ROLLING_DESCALING,0.084
27,HSB_적용,0.066


**<모델 성능>**
- 기본 그래디언트 부스팅 모델은 train과 test 데이터에 대해 각각 100%, 100%의 설명력을 갖는다.
- 의사결정나무에 비해 test 데이터에 대한 성능은 좋지만, 지나친 과대적합의 가능성이 우려된다.

**<변수 중요도>**
- ROLLING_TEMP_T5가 0.612, HSB_미적용이 0.098의 변수 중요도를 갖는다.

### **결론**

- 의사결정나무, 랜덤 포레스트, 그리고 그래디언트 부스팅 예측기를 이용하여 추출한 변수 중요도는 다음과 같다.


| 모델명/순위 |  의사결정나무  |  랜덤 포레스트   |  그래디언트 부스팅  |
|:----------:|:--------------:|:----------------:|:------------------:|
|1|ROLLING_TEMP_T5 (0.592)|ROLLING_TEMP_T5 (0.305)|ROLLING_TEMP_T5 (0.612)|
|2|HSB_미적용 (0.151)     |FUR_SZ_TEMP (0.081) | HSB_미적용 (0.098)         |
|3|FUR_SZ_TEMP (0.094)    |PT_WDTH (0.076)      |ROLLING_DESCALING (0.080)        |
|4|PT_THK (0.086)         |FUR_EXTEMP (0.072)   |FUR_EXTEMP (0.074)       |
|5|ROLLING_DESCALING (0.063)|ROLLING_DESCALING (0.059) |HSB_적용 (0.058)    |

* 3개의 예측 모델 모두 중요하게 고려한 변수는 ROLLING_TEMP_T5, ROLLING_DESCALING이다.
* 압연온도와 압연횟수가 불량제품 생산에 유의한 영향을 미친다고 볼 수 있다.